In [1]:
import os
import mne
import matplotlib.pyplot as plt
import numpy as np
%matplotlib tk

In [2]:
# =============================================================================
# Create MNE Info
# =============================================================================
ch_names = ['SMA', 'leftM1', 'rightM1',
            'STN_R01', 'STN_R12', 'STN_R23',
            'STN_L01', 'STN_L12', 'STN_L23',
            'EMG_R', 'EMG_L',
            'HEOG', 'VEOG',
            'event']
sfreq = 2400
ch_types = ["mag", "mag", "mag",
            "seeg", "seeg", "seeg", "seeg", "seeg", "seeg",
            "emg", "emg",
            "eog", "eog",
            "stim"]

info = mne.create_info(ch_names, sfreq, ch_types, verbose=True)
info["highpass"] = 1
info["lowpass"] = 600

In [3]:
plot_options = {
            "duration": 10,
            "n_channels": 20,
            "scalings": dict(mag=400, seeg=12, eog=100, emg=500, stim=1),
            "color": dict(mag='darkblue', seeg='black', eog='grey', emg='brown',stim='k'),
            "group_by": "original"
            }

In [6]:
# =============================================================================
# LOAD SUBJECTS
# =============================================================================
subj = 11
cond = "off"

path = '../../../data/0_raw/rest/subj'
path_subj = path + f'{subj+1}/{cond}/'  # subjects start at 1
fnames = os.listdir(path_subj)  # load first file only
real_fnames = [fname for fname in fnames if fname[0] != "."] # delete the wird ".DS_Store" files
print(f"Subject {subj+1} has {len(real_fnames)} files.")

raws = []
for fname in real_fnames:
    raws.append(mne.io.read_raw_fieldtrip(path_subj + fname,
                                     info, data_name='data'))

Subject 12 has 1 files.
Creating RawArray with float64 data, n_channels=14, n_times=460800
    Range : 0 ... 460799 =      0.000 ...   192.000 secs
Ready.


# Check Raw data of all channels for each subject entire recording

In [7]:
for i, raw in enumerate(raws):
    fig = raw.plot(title=real_fnames[i], **plot_options)

...mark bad segments...

# Then:

#### Store the annotations

In [14]:
bad_segments = [raw.annotations.copy() for raw in raws]

reject_indices = []
for bad_segs in bad_segments:
    reject_indices.append([(int(seg["onset"]*sfreq), int((seg["onset"] + seg["duration"])*sfreq)) for seg in bad_segs])
reject_indices

[[(119478, 121794),
  (160399, 160797),
  (193009, 196419),
  (246840, 248997),
  (258352, 266147),
  (382747, 383671),
  (431761, 432079),
  (448742, 455970)],
 []]

#### Extract as numpy array

In [15]:
arrays = [arr.get_data() for arr in raws]

for array, reject_index in zip(arrays, reject_indices):
    for reject in reject_index:
        array[:, reject[0]:reject[1]] = np.nan

# Save numpy and fif:

In [16]:
path_fif = f"../../../data/1_raw_clean_fif/rest/subj{subj+1}/{cond}/"

if not os.path.exists(path_fif):
    os.makedirs(path_fif)

for raw, fname in zip(raws, real_fnames):
    raw.save(path_fif + fname[:-4] + "_raw.fif")

Writing /Users/moritzgerster/Documents/Code/Litvak11/data/1_raw_clean_fif/rest/subj1/off/subj1_on_R1_raw.fif
Closing /Users/moritzgerster/Documents/Code/Litvak11/data/1_raw_clean_fif/rest/subj1/off/subj1_on_R1_raw.fif [done]


FileExistsError: Destination file exists. Please use option "overwrite=True" to force overwriting.

In [17]:
path_npy = f"../../../data/1_clean_numpy/rest/subj{subj+1}/{cond}/"

if not os.path.exists(path_npy):
    os.makedirs(path_npy)

for arr, fname in zip(arrays, real_fnames):
    np.save(path_npy + fname[:-4] + "_raw.npy", arr)

### Optional:
... test if annotations are correct:

In [58]:
test = mne.io.read_raw_fif(path_fif + fname[:-4] + "_raw.fif", info)
_ = test.plot(title=fname[:-4], **plot_options)

Opening raw data file ../../data/raw_annotated_fif/rest/subj1/on/subj1_on_R7_raw.fif...
Isotrak not found
    Range : 0 ... 455999 =      0.000 ...   190.000 secs
Ready.
